In [10]:
import ROOT as rt
from tqdm import tqdm

In [11]:
# Define excluded PDG codes
excluded_pdgs = {
    # Quarks
    1, 2, 3, 4, 5, 6,  # up through top
    -1, -2, -3, -4, -5, -6,  # anti-quarks
    
    # Gluons
    21,
    
    # Special codes
    90, 91, 92, 93,  # Cluster, string endpoints, etc
    
    # Diquarks
    1103, 2101, 2103, 2203, 2101, 2103, 2203, 3101, 3103, 3201, 3203,
    -1103, -2101, -2103, -2203, -2101, -2103, -2203, -3101, -3103, -3201, -3203
}

# Define charm hadron PDG codes
charm_hadrons = {
    411, -411,    # D+, D-
    421, -421,    # D0, D0-bar  
    431, -431,    # Ds+, Ds-
    4122, -4122,  # Λc+, Λc-
    4132, -4132,  # Ξc0, Ξc0-bar
    4232, -4232,  # Ξc+, Ξc-
    4332, -4332,  # Ωc0, Ωc0-bar
    # J/psi
    443,
    # Other charm mesons
    423, -423,    # D*(2007)0, D*(2007)0-bar
    413, -413,    # D*(2010)+, D*(2010)-
    433, -433,    # D*_s+, D*_s-
    10411, -10411, # D*_0(2400)+, D*_0(2400)-
    10421, -10421, # D*_0(2400)0, D*_0(2400)0-bar
    10413, -10413, # D_1(2420)+, D_1(2420)-
    10423, -10423, # D_1(2420)0, D_1(2420)0-bar
    20413, -20413, # D_1(2430)+, D_1(2430)-
    20423, -20423, # D_1(2430)0, D_1(2430)0-bar
}

final_charm_hadrons = {
    411, -411,    # D+, D-
    421, -421,    # D0, D0-bar  
    431, -431,    # Ds+, Ds-
    4122, -4122,  # Λc+, Λc-
}

**Let's analyze kinematic correlations for events which have only 2 charmed hadrons. To account for double counting from D\* decay (as proved in charmMultiplicityAnalysis), let's exclude D\* hadrons by referencing final_charm_hadrons.**

In [ ]:
# Open the ROOT file
file = rt.TFile.Open("pt10_10000.root", "READ")
tree = file.Get("EventTree")

# Create histograms for kinematic variables
h_pt = rt.TH1F("h_pt", "p_{T} of \"final\" charm hadrons;p_{T} [GeV/c];Events", 500, 0, 50)
h_y = rt.TH1F("h_y", "Rapidity of \"final\" charm hadrons;y;Events", 30, -5, 5)
h_eta = rt.TH1F("h_eta", "Pseudorapidity of \"final\" charm hadrons;#eta;Events", 30, -5, 5)
h_phi = rt.TH1F("h_phi", "Azimuthal angle of \"final\" charm hadrons;#phi [rad];Events", 40, 0, 2*rt.TMath.Pi())
h_deltaPhi = rt.TH1F("h_deltaPhi", "#Delta#phi between \"final\" charm hadron pairs;#Delta#phi [rad];Events", 20, 0, rt.TMath.Pi())


# Create TClonesArray to hold particles
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", particles)
total_events = tree.GetEntries()
events_with_two_charms = 0

# Loop through all events
for event_idx in tqdm(range(total_events)):
    tree.GetEntry(event_idx)
    
    # Find charm hadrons in this event
    charm_particles = []
    
    # Loop over particles in event
    for j in range(particles.GetEntriesFast()):
        p = particles.At(j)
        pdg = p.GetPdgCode()
        
        # Check if particle is in our final_charm_hadrons list
        if pdg in final_charm_hadrons:
            charm_particles.append(p)
    
    # Only analyze events with exactly 2 charm hadrons
    if len(charm_particles) == 2:
        events_with_two_charms += 1
        
        # Fill individual particle kinematics
        for particle in charm_particles:
            h_pt.Fill(particle.Pt())
            h_y.Fill(particle.Y())
            h_eta.Fill(particle.Eta())
            h_phi.Fill(particle.Phi())
        
        # Calculate and fill delta phi (smaller angle between 0 and pi)
        phi1 = charm_particles[0].Phi()
        phi2 = charm_particles[1].Phi()
        delta_phi = abs(phi1 - phi2)
        
        # Ensure delta_phi is between 0 and π
        while delta_phi > rt.TMath.Pi():
            delta_phi = 2 * rt.TMath.Pi() - delta_phi
            
        h_deltaPhi.Fill(delta_phi)
        print(charm_particles[0].GetPdgCode(), charm_particles[1].GetPdgCode())

# Print statistics
print(f"Total events: {total_events}")
print(f"Events with exactly 2 charm hadrons: {events_with_two_charms} ({100.0*events_with_two_charms/total_events:.2f}%)")

In [24]:
# pT histogram
c_pt = rt.TCanvas("c_pt", "p_{T} Distribution", 800, 600)
h_pt.SetLineColor(rt.kBlue)
h_pt.SetMarkerColor(rt.kBlue)
h_pt.SetMarkerStyle(20)  # Circular marker
h_pt.SetMarkerSize(0.8)
h_pt.SetTitle("p_{T} Distribution of Final Charm Hadrons from 2 event")
h_pt.GetXaxis().SetTitle("p_{T} [GeV/c]")
h_pt.GetYaxis().SetTitle("Events")
h_pt.Draw("EP")  # E for error bars, P for points
c_pt.SetLogy()  # Use log scale for pT distribution
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_pt.Draw()

# Rapidity histogram
c_y = rt.TCanvas("c_y", "Rapidity Distribution from 2 Event", 800, 600)
h_y.SetLineColor(rt.kRed)
h_y.SetMarkerColor(rt.kRed)
h_y.SetMarkerStyle(20)
h_y.SetMarkerSize(0.8)
h_y.SetTitle("Rapidity Distribution of Final Charm Hadrons from 2 event")
h_y.GetXaxis().SetTitle("y")
h_y.GetYaxis().SetTitle("Events")
h_y.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_y.Draw()

# Pseudorapidity histogram
c_eta = rt.TCanvas("c_eta", "Pseudorapidity Distribution", 800, 600)
h_eta.SetLineColor(rt.kGreen+2)
h_eta.SetMarkerColor(rt.kGreen+2)
h_eta.SetMarkerStyle(20)
h_eta.SetMarkerSize(0.8)
h_eta.SetTitle("Pseudorapidity Distribution of Final Charm Hadrons from 2 event")
h_eta.GetXaxis().SetTitle("#eta")
h_eta.GetYaxis().SetTitle("Events")
h_eta.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_eta.Draw()

# Delta phi histogram
c_dphi = rt.TCanvas("c_dphi", "Delta Phi Distribution", 800, 600)
h_deltaPhi.SetLineColor(rt.kMagenta)
h_deltaPhi.SetMarkerColor(rt.kMagenta)
h_deltaPhi.SetMarkerStyle(20)
h_deltaPhi.SetMarkerSize(0.8)
h_deltaPhi.SetMinimum(0)
h_deltaPhi.SetTitle("#Delta#phi Distribution between Final Charm Hadron Pairs from 2 event")
h_deltaPhi.GetXaxis().SetTitle("#Delta#phi [rad]")
h_deltaPhi.GetYaxis().SetTitle("Events")
h_deltaPhi.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_dphi.Draw()

# Phi histogram
c_phi = rt.TCanvas("c_phi", "Phi Distribution", 800, 600)
h_phi.SetLineColor(rt.kOrange+7)
h_phi.SetMarkerColor(rt.kOrange+7)
h_phi.SetMarkerStyle(20)
h_phi.SetMarkerSize(0.8)
h_phi.SetTitle("Azimuthal Angle Distribution of Final Charm Hadrons from 2 event")
h_phi.GetXaxis().SetTitle("#phi [rad]")
h_phi.GetYaxis().SetTitle("Events")
h_phi.GetXaxis().SetRangeUser(0, 100*rt.TMath.Pi())  # Ensure full -π to π range is displayed
h_phi.SetMinimum(0)
h_phi.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_phi.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_pt
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_y
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_eta
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_dphi
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_phi
Warning in <TAxis::TAxis::SetRangeUser>: ulast > fXmax, fXmax is used


WHY IS DPHI SO WEAK IF ITS TWO CHARMS, there should be a correlation here.

note that theres a dip close to 0 pT.

In [25]:
### only close the file if you are done with the analysis, otherwise you can't plot the histograms
file.Close()

Now, lets do this same analysis for all charmed hadrons, not limited to 2 charmed hadrons per event

In [26]:
# Open the ROOT file
file = rt.TFile.Open("pt10_10000.root", "READ")
tree = file.Get("EventTree")

# Create histograms for kinematic variables
h_pt = rt.TH1F("h_pt", "p_{T} of \"final\" charm hadrons;p_{T} [GeV/c];Events/bin", 50, 0, 50)
h_y = rt.TH1F("h_y", "Rapidity of \"final\" charm hadrons;y;Events/bin", 30, -5, 5)
h_eta = rt.TH1F("h_eta", "Pseudorapidity of \"final\" charm hadrons;#eta;Events/bin", 30, -5, 5)
h_deltaPhi = rt.TH1F("h_deltaPhi", "#Delta#phi between trigger and associated charm hadrons;#Delta#phi [rad];Events/bin", 20, 0, rt.TMath.Pi())

# Enable proper error calculation
h_pt.Sumw2()
h_y.Sumw2()
h_eta.Sumw2()
h_deltaPhi.Sumw2()

# Create TClonesArray to hold particles
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", particles)

total_events = tree.GetEntries()
events_with_charms = 0
events_with_multiple_charms = 0
d0_trigger_events = 0

# Loop through all events
for event_idx in tqdm(range(total_events)):
    tree.GetEntry(event_idx)
    
    # Find charm hadrons in this event
    charm_particles = []
    
    # Loop over particles in event
    for j in range(particles.GetEntriesFast()):
        p = particles.At(j)
        pdg = p.GetPdgCode()
        
        # Check if particle is in our final_charm_hadrons list
        if pdg in final_charm_hadrons:
            charm_particles.append(p)
    
    # Skip empty events
    if len(charm_particles) == 0:
        continue
        
    events_with_charms += 1
            
    # Fill individual particle kinematics with proper weight
    weight = 1.0 / len(charm_particles)  # Normalize so each event contributes equally
    for particle in charm_particles:
        h_pt.Fill(particle.Pt(), weight)
        h_y.Fill(particle.Y(), weight)
        h_eta.Fill(particle.Eta(), weight)
    
    # Delta phi calculation only for events with multiple charm hadrons
    if len(charm_particles) > 1:
        events_with_multiple_charms += 1
        
        # Find trigger particle (D0 or first particle)
        trigger_particle = None
        associated_particles = []
        
        # First try to find a D0 (PDG code 421 or -421)
        for p in charm_particles:
            if abs(p.GetPdgCode()) == 421:  # D0 or D0-bar
                trigger_particle = p
                d0_trigger_events += 1
                break
        
        # If no D0 found, use the first particle
        if trigger_particle is None:
            trigger_particle = charm_particles[0]
            associated_particles = charm_particles[1:]
        else:
            # If we found a D0, all other particles are associated
            associated_particles = [p for p in charm_particles if p != trigger_particle]
        
        # Calculate delta phi between trigger and all associated particles
        trigger_phi = trigger_particle.Phi()
        weight_dphi = 1.0 / len(associated_particles)  # Normalize so each event contributes equally
        
        for assoc in associated_particles:
            delta_phi = abs(trigger_phi - assoc.Phi())
            
            # Ensure delta_phi is between 0 and π
            while delta_phi > rt.TMath.Pi():
                delta_phi = 2 * rt.TMath.Pi() - delta_phi
                
            h_deltaPhi.Fill(delta_phi, weight_dphi)

# Print statistics
print(f"Total events processed: {total_events}")
print(f"Events with charm hadrons: {events_with_charms}")
print(f"Events with multiple charm hadrons: {events_with_multiple_charms}")
print(f"Events with D0 trigger: {d0_trigger_events}")
print(f"Average entries per multi-charm event (ΔΦ): {h_deltaPhi.GetEntries()/events_with_multiple_charms:.2f}")

100%|██████████| 10000/10000 [00:22<00:00, 453.43it/s]


Total events processed: 10000
Events with charm hadrons: 3939
Events with multiple charm hadrons: 3849
Events with D0 trigger: 3266
Average entries per multi-charm event (ΔΦ): 1.80


In [27]:
# pT histogram
c_pt = rt.TCanvas("c_pt", "p_{T} Distribution", 800, 600)
h_pt.SetLineColor(rt.kBlue)
h_pt.SetMarkerColor(rt.kBlue)
h_pt.SetMarkerStyle(20)
h_pt.SetMarkerSize(0.8)
h_pt.SetTitle("p_{T} Distribution of Final Charm Hadrons")
h_pt.GetXaxis().SetTitle("p_{T} [GeV/c]")
h_pt.GetYaxis().SetTitle("Events/bin")
h_pt.Draw("EP")  # E for error bars, P for points
c_pt.SetLogy()  # Use log scale for pT distribution
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_pt.Draw()

# Rapidity histogram
c_y = rt.TCanvas("c_y", "Rapidity Distribution", 800, 600)
h_y.SetLineColor(rt.kRed)
h_y.SetMarkerColor(rt.kRed)
h_y.SetMarkerStyle(20)
h_y.SetMarkerSize(0.8)
h_y.SetTitle("Rapidity Distribution of Final Charm Hadrons")
h_y.GetXaxis().SetTitle("y")
h_y.GetYaxis().SetTitle("Events/bin")
h_y.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_y.Draw()

# Pseudorapidity histogram
c_eta = rt.TCanvas("c_eta", "Pseudorapidity Distribution", 800, 600)
h_eta.SetLineColor(rt.kGreen+2)
h_eta.SetMarkerColor(rt.kGreen+2)
h_eta.SetMarkerStyle(20)
h_eta.SetMarkerSize(0.8)
h_eta.SetTitle("Pseudorapidity Distribution of Final Charm Hadrons")
h_eta.GetXaxis().SetTitle("#eta")
h_eta.GetYaxis().SetTitle("Events/bin")
h_eta.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_eta.Draw()

# Delta phi histogram
c_dphi = rt.TCanvas("c_dphi", "Delta Phi Distribution", 800, 600)
h_deltaPhi.SetLineColor(rt.kMagenta)
h_deltaPhi.SetMarkerColor(rt.kMagenta)
h_deltaPhi.SetMarkerStyle(20)
h_deltaPhi.SetMinimum(0)
h_deltaPhi.SetMarkerSize(0.8)
h_deltaPhi.SetTitle("#Delta#phi Distribution between Trigger and Associated Charm Hadrons")
h_deltaPhi.GetXaxis().SetTitle("#Delta#phi [rad]")
h_deltaPhi.GetYaxis().SetTitle("Events/bin")
h_deltaPhi.Draw("EP")
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
c_dphi.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_pt
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_y
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_eta
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_dphi


maybe we can increase event number to then see if its statistical error?

In [28]:
# Only close file after you are done with analysis
file.Close()

In [3]:
# Open the ROOT file
file = rt.TFile.Open("pt10_10000.root", "READ")
tree = file.Get("EventTree")

# Create 2D correlation histograms
h_pt_corr = rt.TH2F("h_pt_corr", "p_{T} Correlation;p_{T,1} [GeV/c];p_{T,2} [GeV/c]", 50, 0, 50, 50, 0, 50)
h_eta_corr = rt.TH2F("h_eta_corr", "#eta Correlation;#eta_{1};#eta_{2}", 30, -5, 5, 30, -5, 5)
h_y_corr = rt.TH2F("h_y_corr", "y Correlation;y_{1};y_{2}", 30, -5, 5, 30, -5, 5)

# Keep histograms in memory
h_pt_corr.SetDirectory(0)
h_eta_corr.SetDirectory(0)
h_y_corr.SetDirectory(0)

# Create TClonesArray to hold particles
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", particles)

total_events = tree.GetEntries()
print(f"Total events in tree: {total_events}")

events_with_multiple_charms = 0
d0_trigger_events = 0

# Debug counters
particles_checked = 0
charm_particles_found = 0

# Loop through all events
for event_idx in tqdm(range(total_events)):
    tree.GetEntry(event_idx)
    
    # Find charm hadrons in this event
    charm_particles = []
    
    # Debug: print first event's particles
    if event_idx == 0:
        print(f"\nChecking first event with {particles.GetEntriesFast()} particles")
    
    # Loop over particles in event
    for j in range(particles.GetEntriesFast()):
        particles_checked += 1
        p = particles.At(j)
        pdg = p.GetPdgCode()
        
        # Debug first event
        if event_idx == 0:
            print(f"Particle {j}: PDG = {pdg}")
        
        # Check if particle is in our final_charm_hadrons list
        if pdg in final_charm_hadrons:
            charm_particles_found += 1
            charm_particles.append(p)
    
    # Skip if less than 2 charm particles
    if len(charm_particles) < 2:
        continue
        
    events_with_multiple_charms += 1
    
    # Debug: print first event with multiple charms
    if events_with_multiple_charms == 1:
        print(f"\nFirst event with multiple charms found (Event {event_idx})")
        print(f"Number of charm particles: {len(charm_particles)}")
        for i, p in enumerate(charm_particles):
            print(f"Charm {i}: PDG = {p.GetPdgCode()}, pT = {p.Pt():.2f}, eta = {p.Eta():.2f}")
    
    # Find trigger particle (D0 or first particle)
    trigger_particle = None
    associated_particles = []
    
    # First try to find a D0 (PDG code 421 or -421)
    for p in charm_particles:
        if abs(p.GetPdgCode()) == 421:  # D0 or D0-bar
            trigger_particle = p
            d0_trigger_events += 1
            break
    
    # If no D0 found, use the first particle
    if trigger_particle is None:
        trigger_particle = charm_particles[0]
        associated_particles = charm_particles[1:]
    else:
        # If we found a D0, all other particles are associated
        associated_particles = [p for p in charm_particles if p != trigger_particle]
    
    # Get trigger kinematics
    trigger_pt = trigger_particle.Pt()
    trigger_eta = trigger_particle.Eta()
    trigger_y = trigger_particle.Y()
    
    # Fill correlations with associated particles
    weight = 1.0 / len(associated_particles)  # Weight to normalize per event
    for assoc in associated_particles:
        h_pt_corr.Fill(trigger_pt, assoc.Pt(), weight)
        h_eta_corr.Fill(trigger_eta, assoc.Eta(), weight)
        h_y_corr.Fill(trigger_y, assoc.Y(), weight)

# Print detailed statistics
print("\nFinal Statistics:")
print(f"Total events processed: {total_events}")
print(f"Total particles checked: {particles_checked}")
print(f"Total charm particles found: {charm_particles_found}")
print(f"Events with multiple charm hadrons: {events_with_multiple_charms}")
print(f"Events with D0 trigger: {d0_trigger_events}")
print(f"\nHistogram Statistics:")
print(f"pT correlation entries: {h_pt_corr.GetEntries()}")
print(f"eta correlation entries: {h_eta_corr.GetEntries()}")
print(f"y correlation entries: {h_y_corr.GetEntries()}")

# Now draw the histograms
# Set up nice color palette
rt.gStyle.SetPalette(rt.kBird)
rt.gStyle.SetNumberContours(99)

# pT correlation
c_pt_corr = rt.TCanvas("c_pt_corr", "p_{T} Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetRightMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
h_pt_corr.SetTitle("p_{T} Correlation of Trigger and Associated Charm Hadrons")
h_pt_corr.Draw("COLZ")
c_pt_corr.Draw()

# Eta correlation
c_eta_corr = rt.TCanvas("c_eta_corr", "#eta Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetRightMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
h_eta_corr.SetTitle("#eta Correlation of Trigger and Associated Charm Hadrons")
h_eta_corr.Draw("COLZ")
c_eta_corr.Draw()

# Rapidity correlation
c_y_corr = rt.TCanvas("c_y_corr", "y Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15)
rt.gPad.SetRightMargin(0.15)
rt.gPad.SetBottomMargin(0.15)
h_y_corr.SetTitle("Rapidity Correlation of Trigger and Associated Charm Hadrons")
h_y_corr.Draw("COLZ")
c_y_corr.Draw()


Total events in tree: 10000


  0%|          | 16/10000 [00:00<01:02, 159.00it/s]


Checking first event with 453 particles
Particle 0: PDG = 90
Particle 1: PDG = 2212
Particle 2: PDG = 2212
Particle 3: PDG = 21
Particle 4: PDG = 21
Particle 5: PDG = 21
Particle 6: PDG = 21
Particle 7: PDG = 21
Particle 8: PDG = 21
Particle 9: PDG = 21
Particle 10: PDG = 21
Particle 11: PDG = 21
Particle 12: PDG = 21
Particle 13: PDG = 21
Particle 14: PDG = 21
Particle 15: PDG = 21
Particle 16: PDG = 21
Particle 17: PDG = 21
Particle 18: PDG = 21
Particle 19: PDG = 21
Particle 20: PDG = 21
Particle 21: PDG = 21
Particle 22: PDG = 21
Particle 23: PDG = 21
Particle 24: PDG = 21
Particle 25: PDG = 21
Particle 26: PDG = 21
Particle 27: PDG = 21
Particle 28: PDG = 21
Particle 29: PDG = 21
Particle 30: PDG = 21
Particle 31: PDG = 21
Particle 32: PDG = 21
Particle 33: PDG = -1
Particle 34: PDG = 1
Particle 35: PDG = 21
Particle 36: PDG = 21
Particle 37: PDG = 21
Particle 38: PDG = 21
Particle 39: PDG = 21
Particle 40: PDG = 21
Particle 41: PDG = 21
Particle 42: PDG = -1
Particle 43: PDG = 2

100%|██████████| 10000/10000 [00:23<00:00, 430.70it/s]



Final Statistics:
Total events processed: 10000
Total particles checked: 12749620
Total charm particles found: 10854
Events with multiple charm hadrons: 3849
Events with D0 trigger: 3266

Histogram Statistics:
pT correlation entries: 6915.0
eta correlation entries: 6915.0
y correlation entries: 6915.0


look at slices for each 2d plot. for example, in the pT, if the pt of d0 is in bin 0-1, how would the distrobution of the assoc particles look?

look at the peak mean value, the size shape, if the peak is shifting, by wht factor? does that factor change?

add phi correlation plot so i can see where exactly assoc particle is. if phi is also directly, then the trigger and assoc is likely in the exact same place.

try to get the same conclusions to find where to assoc particle is using dphi, as it give sthe same info.

then do a 2D correlation between dphi and deta to get the loc in one plot

do the correlation for 2 charm events, then 3, etc. all odd, all even. might be unlcear at higher events bc les data counts

In [4]:
file.Close

In [9]:
# Open the ROOT file
file = rt.TFile.Open("pt10_10000.root", "READ")
tree = file.Get("EventTree")

# Create 2D correlation histograms
h_pt_corr = rt.TH2F(
    "h_pt_corr",
    "p_{T} Correlation;p_{T,1} [GeV/c];p_{T,2} [GeV/c]",
    50, 0, 50,
    50, 0, 50
)
h_eta_corr = rt.TH2F(
    "h_eta_corr",
    "#eta Correlation;#eta_{1};#eta_{2}",
    30, -5, 5,
    30, -5, 5
)
h_y_corr = rt.TH2F(
    "h_y_corr",
    "y Correlation;y_{1};y_{2}",
    30, -5, 5,
    30, -5, 5
)
# **New**: φ correlation
h_phi_corr = rt.TH2F(
    "h_phi_corr",
    "#phi Correlation;#phi_{trigger} [rad];#phi_{assoc} [rad]",
    36, 0, 2 * 3.15,
    36, 0, 2 * 3.15
)

# Keep histograms in memory
for h in (h_pt_corr, h_eta_corr, h_y_corr, h_phi_corr):
    h.SetDirectory(0)

# Create TClonesArray to hold particles
particles = rt.TClonesArray("TParticle")
tree.SetBranchAddress("Particles", particles)

total_events = tree.GetEntries()
print(f"Total events in tree: {total_events}")

events_with_multiple_charms = 0
d0_trigger_events = 0

# Debug counters
particles_checked = 0
charm_particles_found = 0

# Loop through all events
for event_idx in tqdm(range(total_events)):
    tree.GetEntry(event_idx)
    
    # Find charm hadrons in this event
    charm_particles = []
    
    # Debug: print first event's particles
    if event_idx == 0:
        print(f"\nChecking first event with {particles.GetEntriesFast()} particles")
    
    # Loop over particles in event
    for j in range(particles.GetEntriesFast()):
        particles_checked += 1
        p = particles.At(j)
        pdg = p.GetPdgCode()
        
        # Debug first event
        if event_idx == 0:
            print(f"Particle {j}: PDG = {pdg}")
        
        # Check if particle is in our final_charm_hadrons list
        if pdg in final_charm_hadrons:
            charm_particles_found += 1
            charm_particles.append(p)
    
    # Skip if less than 2 charm particles
    if len(charm_particles) < 2:
        continue
        
    events_with_multiple_charms += 1
    
    # Debug: print first event with multiple charms
    if events_with_multiple_charms == 1:
        print(f"\nFirst event with multiple charms found (Event {event_idx})")
        print(f"Number of charm particles: {len(charm_particles)}")
        for i, p in enumerate(charm_particles):
            print(f"Charm {i}: PDG = {p.GetPdgCode()}, pT = {p.Pt():.2f}, eta = {p.Eta():.2f}")
    
    # Find trigger particle (D0 or first particle)
    trigger_particle = None
    associated_particles = []
    
    # First try to find a D0 (PDG code 421 or -421)
    for p in charm_particles:
        if abs(p.GetPdgCode()) == 421:  # D0 or D0-bar
            trigger_particle = p
            d0_trigger_events += 1
            break
    
    # If no D0 found, use the first particle
    if trigger_particle is None:
        trigger_particle = charm_particles[0]
        associated_particles = charm_particles[1:]
    else:
        # If we found a D0, all other particles are associated
        associated_particles = [p for p in charm_particles if p != trigger_particle]
    
    # Get trigger kinematics
    trigger_pt  = trigger_particle.Pt()
    trigger_eta = trigger_particle.Eta()
    trigger_y   = trigger_particle.Y()
    trigger_phi = trigger_particle.Phi()    # <-- new
    
    # Fill correlations with associated particles
    weight = 1.0 / len(associated_particles)  # normalize per event
    for assoc in associated_particles:
        h_pt_corr .Fill(trigger_pt,  assoc.Pt(),   weight)
        h_eta_corr.Fill(trigger_eta, assoc.Eta(),  weight)
        h_y_corr  .Fill(trigger_y,   assoc.Y(),    weight)
        h_phi_corr.Fill(trigger_phi, assoc.Phi(),  weight)  # <-- new

# Print detailed statistics
print("\nFinal Statistics:")
print(f"Total events processed: {total_events}")
print(f"Total particles checked: {particles_checked}")
print(f"Total charm particles found: {charm_particles_found}")
print(f"Events with multiple charm hadrons: {events_with_multiple_charms}")
print(f"Events with D0 trigger: {d0_trigger_events}")
print(f"\nHistogram Statistics:")
print(f"pT correlation entries: {h_pt_corr.GetEntries()}")
print(f"eta correlation entries: {h_eta_corr.GetEntries()}")
print(f"y correlation entries: {h_y_corr.GetEntries()}")
print(f"phi correlation entries: {h_phi_corr.GetEntries()}")  # <-- new

# Now draw the histograms
rt.gStyle.SetPalette(rt.kBird)
rt.gStyle.SetNumberContours(99)

# pT correlation
c_pt_corr = rt.TCanvas("c_pt_corr", "p_{T} Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15); rt.gPad.SetRightMargin(0.15); rt.gPad.SetBottomMargin(0.15)
h_pt_corr.SetTitle("p_{T} Correlation of Trigger and Associated Charm Hadrons")
h_pt_corr.Draw("COLZ")
c_pt_corr.Draw()

# Eta correlation
c_eta_corr = rt.TCanvas("c_eta_corr", "#eta Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15); rt.gPad.SetRightMargin(0.15); rt.gPad.SetBottomMargin(0.15)
h_eta_corr.SetTitle("#eta Correlation of Trigger and Associated Charm Hadrons")
h_eta_corr.Draw("COLZ")
c_eta_corr.Draw()

# Rapidity correlation
c_y_corr = rt.TCanvas("c_y_corr", "y Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15); rt.gPad.SetRightMargin(0.15); rt.gPad.SetBottomMargin(0.15)
h_y_corr.SetTitle("Rapidity Correlation of Trigger and Associated Charm Hadrons")
h_y_corr.Draw("COLZ")
c_y_corr.Draw()

# Phi correlation (new)
c_phi_corr = rt.TCanvas("c_phi_corr", "#phi Correlation", 800, 700)
rt.gPad.SetLeftMargin(0.15); rt.gPad.SetRightMargin(0.15); rt.gPad.SetBottomMargin(0.15)
h_phi_corr.SetTitle("Azimuthal (φ) Correlation of Trigger and Associated Charm Hadrons")
h_phi_corr.Draw("COLZ")
c_phi_corr.Draw()

Total events in tree: 10000


  0%|          | 29/10000 [00:00<00:34, 286.07it/s]


Checking first event with 453 particles
Particle 0: PDG = 90
Particle 1: PDG = 2212
Particle 2: PDG = 2212
Particle 3: PDG = 21
Particle 4: PDG = 21
Particle 5: PDG = 21
Particle 6: PDG = 21
Particle 7: PDG = 21
Particle 8: PDG = 21
Particle 9: PDG = 21
Particle 10: PDG = 21
Particle 11: PDG = 21
Particle 12: PDG = 21
Particle 13: PDG = 21
Particle 14: PDG = 21
Particle 15: PDG = 21
Particle 16: PDG = 21
Particle 17: PDG = 21
Particle 18: PDG = 21
Particle 19: PDG = 21
Particle 20: PDG = 21
Particle 21: PDG = 21
Particle 22: PDG = 21
Particle 23: PDG = 21
Particle 24: PDG = 21
Particle 25: PDG = 21
Particle 26: PDG = 21
Particle 27: PDG = 21
Particle 28: PDG = 21
Particle 29: PDG = 21
Particle 30: PDG = 21
Particle 31: PDG = 21
Particle 32: PDG = 21
Particle 33: PDG = -1
Particle 34: PDG = 1
Particle 35: PDG = 21
Particle 36: PDG = 21
Particle 37: PDG = 21
Particle 38: PDG = 21
Particle 39: PDG = 21
Particle 40: PDG = 21
Particle 41: PDG = 21
Particle 42: PDG = -1
Particle 43: PDG = 2

100%|██████████| 10000/10000 [00:22<00:00, 446.12it/s]



Final Statistics:
Total events processed: 10000
Total particles checked: 12749620
Total charm particles found: 10854
Events with multiple charm hadrons: 3849
Events with D0 trigger: 3266

Histogram Statistics:
pT correlation entries: 6915.0
eta correlation entries: 6915.0
y correlation entries: 6915.0
phi correlation entries: 6915.0


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_pt_corr
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_eta_corr
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_y_corr
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_phi_corr


make 2d correlation for just 2 charmed filtering, it should be sharper

make an eta cut for the dphi -- if the two particles have an deta between -1 and 1, they are more likely to be correlated -> then calc dphi. or cut do ETa between -1 and 1, then do an Deta cut? 